In [46]:
import copy
# TODO: investigate death works well and internal death stats
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# TODO: Collect internal statistics of history of Neurons at their death point (like: values, how fast changed, etc.)
# "batch norm analog" - i.e. collector neuron - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# Separate livenet core and high-level api?
# TODO: make amount of dangle/useless neurons updated during Neuron creation/connection?

import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import livenet
importlib.reload(livenet)
core = livenet.core
import simple_log
simple_log.level = simple_log.LogLevel.INFO
from simple_log import LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
livenet.core.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.2.0+cu121


'%.4f'

In [7]:
downscale = (8, 8)
test_x, test_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_test(), downscale=downscale)
train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_train(), downscale=downscale)


0.4%

100.0%


Extracting /home/spometun/datasets/research/cifar-10-python.tar.gz to /home/spometun/datasets/research
Files already downloaded and verified


In [9]:
context = core.livenet.Context()
network = livenet.nets.create_perceptron(train_x.shape[1], 21, 10, context)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=10)

LiveNet


In [48]:
network.context.regularization_l1 = 0.001
optimizer.learning_rate = 0.001
network.context.learning_rate = 0.001
trainer.step(1000)


Iˈ0.000 2.546 = 2.381+0.164 params=202 livenet/net_trainer.py:93
Iˈ0.424 2.527 = 2.362+0.164 params=202 livenet/net_trainer.py:93
Iˈ0.823 2.561 = 2.397+0.164 params=202 livenet/net_trainer.py:93
Iˈ1.219 2.542 = 2.378+0.164 params=202 livenet/net_trainer.py:93
Iˈ1.621 2.553 = 2.389+0.165 params=202 livenet/net_trainer.py:93
Iˈ2.021 2.545 = 2.381+0.164 params=202 livenet/net_trainer.py:93
Iˈ2.418 2.536 = 2.372+0.164 params=202 livenet/net_trainer.py:93
Iˈ2.822 2.574 = 2.410+0.164 params=202 livenet/net_trainer.py:93
Iˈ3.226 2.536 = 2.372+0.164 params=202 livenet/net_trainer.py:93
Iˈ3.628 2.570 = 2.406+0.164 params=202 livenet/net_trainer.py:93
Iˈ4.029 2.518 = 2.354+0.164 params=202 livenet/net_trainer.py:93
Iˈ4.429 2.552 = 2.388+0.164 params=202 livenet/net_trainer.py:93
Iˈ4.833 2.542 = 2.378+0.164 params=202 livenet/net_trainer.py:93
Iˈ5.235 2.577 = 2.413+0.164 params=202 livenet/net_trainer.py:93
Iˈ5.641 2.543 = 2.380+0.163 params=202 livenet/net_trainer.py:93
Iˈ6.047 2.514 = 2.350+0.1

In [45]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff)}"


'accuracy 0.4117'

In [ ]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.grid()
#plt.plot(values)

accum = core.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()


In [ ]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")


In [ ]:
network._parameters


In [ ]:
core.livenet.export_onnx(network, "/home/spometun/model.onnx")
